# Sentinel 2 AOI Cloud-free Compositing tool 
Author - Jeff Anderson

The purpose of this tool is to:
<ul>
    <li>Authenticate and link a google account with GeeMap, so that the image can download straight to your google drive</li>
    <li>Create a cloud-free composite image for the area of interest (aoi) from images within the date range</li>
    <li>Download the composite image to your google drive</li>
</ul>

With additional time:
<ul>
    <li>Make it so that it makes a composite image from images representative closest to a central date, rather that earliest within a date range</li>
    <li>Have it do multiple years at once</li>
    <li>Add in the option of selecting several areas of interest</li>
</ul>

This first block loads required packages and authenticates and initiallizes google earth engine (gee or ee) 

In [ ]:
import ee
import geemap
import os
from datetime import datetime

# Authenticate and initialize Earth Engine using geemap
geemap.ee_initialize()
ee.Authenticate()

This block: 
<ul>
    <li>Defines the area of interest with a bounding box - [this tool helps create a bounding box](http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000)</li>
    <li>Creates a mask for clouds</li>
    <li>Creates a composite image to cover the full area of interest, and fills holes which were masked out due to clouds</li>
    <li>Sets visable parameters and displayes the image for confirmation</li>
</ul>



In [ ]:
# Define an area of interest (AOI) as a GeoJSON-like dictionary
aoi = ee.Geometry.Polygon([
    [
        [-125.195, 54.01],
        [-125.195, 54.07],
        [-125.03, 54.07],
        [-125.03, 54.01],
    ]
])

# Define a date range
start_date = '2022-06-20'
end_date = '2022-07-18'

# Create an interactive map
Map = geemap.Map(center=[54.043223, -125.117145], zoom=12)

# Function to mask clouds using the SCL band
def mask_clouds(image):
    scl = image.select('SCL')
    # Values indicating clear conditions
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return image.updateMask(mask)

# Filter Sentinel-2 image collection for the date range and AOI
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi) \
    .map(mask_clouds) \
    .median()  # Create a median composite image

# Clip the composite image to the AOI
composite_image = collection.clip(aoi)



# Visualize the image on the map
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue bands
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

Map.addLayer(composite_image, vis_params, 'Sentinel-2 Composite Image')

# Display the map
Map

This block: 
<ul>
    <li>Strip date (or in this case year) to append to filename</li>
    <li>Export the raster to your google drive</li>
</ul>


In [ ]:
# Get the date range as the year for the filename
start_year = datetime.strptime(start_date, '%Y-%m-%d').year
end_year = datetime.strptime(end_date, '%Y-%m-%d').year
year_range = f'{start_year}-{end_year}'

# Define the output file parameters
description = f'Endako_Sent2_{year_range}'
folder = 'EarthEngineImages'  # Google Drive folder to save the image
fileNamePrefix = f'Endako_Sent2_{year_range}'

# Export the image to Google Drive
geemap.ee_export_image_to_drive(
    composite_image, 
    description=description, 
    folder=folder, 
    fileNamePrefix=fileNamePrefix, 
    scale=10, 
    region=aoi
)

print(f'Task started for exporting image to Google Drive: {description}')